In [21]:
import plotly.express as px
from mainnet_launch.constants import *
from mainnet_launch.pages.rebalance_events.rebalance_events import fetch_rebalance_events_df
from mainnet_launch.database.schema.postgres_operations import get_full_table_as_df
from mainnet_launch.database.schema.full import *
import plotly.io as pio

pio.templates.default = None

autopool = AUTO_DOLA
df = fetch_rebalance_events_df(autopool)
# you need to rename the spot swap cost values

# px.scatter(df, x='swap_cost_in_bps_of_value_out', y='safe_slippage_bps', color='long_move_name')

2025-06-05 15:52:28.295 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:52:28.296 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:52:28.297 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:52:28.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:52:28.672 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:52:28.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:52:28.674 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:52:28.676 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [41]:
df[df["swap_cost_in_bps_of_NAV"] > 12].T

datetime,2025-06-03 04:34:59+00:00
tx_hash,0x272e39d795a4a38e6cc22ec9e54deb92fa068c83304d...
autopool_vault_address,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d
chain_id,1
rebalance_file_path,rebalance_plan_1748925231_0x79eB84B5E30Ef2481c...
destination_out,0x79eB84B5E30Ef2481c8f00fD0Aa7aAd6Ac0AA54d
destination_in,0x116b3e86f2b04c21605c5fC4b95Ba6A82dBF8F3E
quantity_out,507586.651647
quantity_in,448213.886034
safe_value_out,507586.651647
safe_value_in,507510.513649


In [23]:
df[df["tokens_move_name"] == "('USDC',) -> ('USDC',)"]["move_name"].value_counts().index

Index([], dtype='object', name='move_name')

In [24]:
df[df["tokens_move_name"] == "('USDC',) -> ('USDC',)"]["move_name"].value_counts().index

Index([], dtype='object', name='move_name')

In [25]:
df["long_move_name"] = df["move_name"] + "   " + df["tokens_move_name"]
df["long_move_name"].value_counts()

long_move_name
autoDOLA -> reusdsdola   ('DOLA',) -> ('reUSD', 'sDOLA')    20
autoDOLA -> savedola   ('DOLA',) -> ('sDOLA', 'scrvUSD')     8
autoDOLA -> sDOLA   ('DOLA',) -> ('sDOLA',)                  5
autoDOLA -> DOLA-sUSDe   ('DOLA',) -> ('sUSDe', 'DOLA')      2
Name: count, dtype: int64

In [26]:
# spot_value_in_solver_change is not correct in BASE_USD

In [27]:
(df["spot_value_in_solver_change"] > 0).value_counts(normalize=True)

spot_value_in_solver_change
True     0.942857
False    0.057143
Name: proportion, dtype: float64

In [28]:
px.histogram(df["spot_value_in_solver_change"])

In [29]:
df["long_move_name"] = df["move_name"] + "   " + df["tokens_move_name"]
cond = df["move_name"].isin(["autoDOLA -> sDOLA", "baseUSD -> fUSDC", "baseUSD -> smUSDC", "baseUSD -> mwUSDC"])
cond = cond | (df["tokens_move_name"] == "('USDC',) -> ('USDC',)")
df["adjusted_spot_swap_cost"] = df["spot_value_in_solver_change"].where(
    cond, df["spot_swap_cost"] - df["spot_value_in_solver_change"]
)
df["adjusted_spot_swap_cost_in_bps_of_value_out"] = 10_000 * df["adjusted_spot_swap_cost"] / df["spot_value_out"]
df["adjusted_spot_swap_cost_in_bps_of_NAV"] = 10_000 * df["adjusted_spot_swap_cost"] / df["total_nav"]

In [30]:
px.scatter(df, x="adjusted_spot_swap_cost_in_bps_of_value_out", y="spot_value_out", color="long_move_name")

In [31]:
expensive_rebalances = df[
    df["adjusted_spot_swap_cost_in_bps_of_value_out"]
    > df["adjusted_spot_swap_cost_in_bps_of_value_out"].describe()["75%"]
]
expensive_rebalances.groupby("long_move_name")["adjusted_spot_swap_cost_in_bps_of_value_out"].describe().round(2).T.T
# 3bps is the 75th percentile of swap cost on baseUSD

,count,mean,std,min,25%,50%,75%,max
long_move_name,,,,,,,,
"autoDOLA -> DOLA-sUSDe ('DOLA',) -> ('sUSDe', 'DOLA')",2.0,11.72,8.13,5.97,8.84,11.72,14.59,17.46
"autoDOLA -> reusdsdola ('DOLA',) -> ('reUSD', 'sDOLA')",6.0,7.31,4.63,3.93,4.30,4.66,10.15,14.44
"autoDOLA -> savedola ('DOLA',) -> ('sDOLA', 'scrvUSD')",1.0,6.32,NaN,6.32,6.32,6.32,6.32,6.32


In [32]:
all_time_performance = df.groupby("long_move_name")[["adjusted_spot_swap_cost", "spot_value_out"]].sum()

all_time_performance["average_bps"] = (
    10_000 * all_time_performance["adjusted_spot_swap_cost"] / all_time_performance["spot_value_out"]
)

all_time_performance["spot_value_out_thousands"] = all_time_performance["spot_value_out"] / 1_000
all_time_performance[["spot_value_out_thousands", "adjusted_spot_swap_cost", "average_bps"]].round()

,spot_value_out_thousands,adjusted_spot_swap_cost,average_bps
long_move_name,,,
"autoDOLA -> DOLA-sUSDe ('DOLA',) -> ('sUSDe', 'DOLA')",135.0,167.0,12.0
"autoDOLA -> reusdsdola ('DOLA',) -> ('reUSD', 'sDOLA')",1009.0,324.0,3.0
"autoDOLA -> sDOLA ('DOLA',) -> ('sDOLA',)",1624.0,244.0,1.0
"autoDOLA -> savedola ('DOLA',) -> ('sDOLA', 'scrvUSD')",499.0,57.0,1.0


In [33]:
df.groupby("long_move_name")["adjusted_spot_swap_cost"].sum().sort_values(ascending=False)

long_move_name
autoDOLA -> reusdsdola   ('DOLA',) -> ('reUSD', 'sDOLA')    323.603810
autoDOLA -> sDOLA   ('DOLA',) -> ('sDOLA',)                 243.562420
autoDOLA -> DOLA-sUSDe   ('DOLA',) -> ('sUSDe', 'DOLA')     166.799237
autoDOLA -> savedola   ('DOLA',) -> ('sDOLA', 'scrvUSD')     56.682970
Name: adjusted_spot_swap_cost, dtype: float64

In [34]:
df[df["move_name"] == "autoDOLA -> sDOLA"]["spot_swap_cost"]

datetime
2025-06-03 04:34:59+00:00    1344.236960
2025-06-03 05:52:47+00:00     265.951329
2025-06-03 14:04:35+00:00     197.792393
2025-06-04 16:04:23+00:00    -151.587377
2025-06-05 06:04:23+00:00     112.064746
Name: spot_swap_cost, dtype: float64

In [35]:
(df["move_name"] == "autoDOLA -> sDOLA").value_counts()

move_name
False    30
True      5
Name: count, dtype: int64

In [36]:
df.groupby("long_move_name")["swap_cost_in_bps_of_value_out"].describe().T.round(2)

long_move_name,"autoDOLA -> DOLA-sUSDe ('DOLA',) -> ('sUSDe', 'DOLA')","autoDOLA -> reusdsdola ('DOLA',) -> ('reUSD', 'sDOLA')","autoDOLA -> sDOLA ('DOLA',) -> ('sDOLA',)","autoDOLA -> savedola ('DOLA',) -> ('sDOLA', 'scrvUSD')"
count,2.00,20.00,5.00,8.00
mean,-19.87,5.11,11.36,2.88
std,29.96,3.61,10.69,3.07
min,-41.06,0.93,-2.85,-3.01
25%,-30.47,3.36,7.77,2.13
50%,-19.87,4.55,10.48,2.79
75%,-9.28,5.51,14.94,4.02
max,1.31,15.94,26.48,7.82


In [37]:
df.groupby("long_move_name")["safe_slippage_bps"].describe().T.round(2)

long_move_name,"autoDOLA -> DOLA-sUSDe ('DOLA',) -> ('sUSDe', 'DOLA')","autoDOLA -> reusdsdola ('DOLA',) -> ('reUSD', 'sDOLA')","autoDOLA -> sDOLA ('DOLA',) -> ('sDOLA',)","autoDOLA -> savedola ('DOLA',) -> ('sDOLA', 'scrvUSD')"
count,2.00,20.00,5.0,8.00
mean,-19.91,6.29,1.5,2.64
std,28.59,4.73,0.0,2.70
min,-40.13,-4.99,1.5,-2.75
25%,-30.02,4.20,1.5,2.08
50%,-19.91,7.09,1.5,2.60
75%,-9.80,8.29,1.5,3.68
max,0.31,16.67,1.5,6.72


In [38]:
df.co

AttributeError: 'DataFrame' object has no attribute 'co'

In [ ]:
df[df["tx_hash"] == "0x272e39d795a4a38e6cc22ec9e54deb92fa068c83304d5a6e7054c9d68702d21d"].T
# expected to have 0 slippage

In [ ]:
from multicall import Call
from mainnet_launch.data_fetching.get_state_by_block import *

In [ ]:
AUTO_USD_ROOT_PRICE_ORACLE = "0xdB8747a396D75D576Dc7a10bb6c8F02F4a3C20f1"
DOLA = "0x865377367054516e17014CcdED1e7d814EDC9ce4"
sDOLA = "0xb45ad160634c528Cc3D2926d9807104FA3157305"

sDOLA_exchange_rate = Call(
    sDOLA,
    ["convertToAssets(uint256)(uint256)", int(1e18)],
    [("sDOLA_exchange_rate", safe_normalize_with_bool_success)],
)

# drop the parentheses on DOLA – it’s just a string here, not a function
sDOLA_spot_price_call = Call(
    AUTO_USD_ROOT_PRICE_ORACLE,
    ["getSpotPriceInQuote(address,address,address)(uint256)", sDOLA, sDOLA, DOLA],
    [("sDOLA_spot_price", safe_normalize_with_bool_success)],
)

autoDOLA_solver = "0x2795B6a09df08891EF21f7153f4799cFBe3Ae6b6"

dola_autopool_bal_call = Call(
    DOLA,
    ["balanceOf(address)(uint256)", AUTO_DOLA.autopool_eth_addr],
    [("autopool_bal", safe_normalize_with_bool_success)],
)

dola_solver_bal_call = Call(
    DOLA,
    ["balanceOf(address)(uint256)", autoDOLA_solver],
    [("solver_bal", safe_normalize_with_bool_success)],
)


before = get_state_by_one_block(
    [sDOLA_exchange_rate, sDOLA_spot_price_call, dola_autopool_bal_call, dola_solver_bal_call],
    22621780 - 1,  # block_number - 1
    ETH_CHAIN,
)

after = get_state_by_one_block(
    [sDOLA_exchange_rate, sDOLA_spot_price_call, dola_autopool_bal_call, dola_solver_bal_call],
    22621780,  # block_number - 1
    ETH_CHAIN,
)
{"before": before, "after": after}

In [ ]:
blocks = build_blocks_to_use(ETH_CHAIN)

sDOLA_price_df = get_raw_state_by_blocks(
    [sDOLA_exchange_rate, sDOLA_spot_price_call], blocks, ETH_CHAIN, include_block_number=True
)

px.line(sDOLA_price_df)

In [ ]:
blocks_to_check = [b for b in range(22584692, 22640079, 500)]
sDOLA_price_df = get_raw_state_by_blocks([sDOLA_exchange_rate, sDOLA_spot_price_call], blocks_to_check, ETH_CHAIN)

px.line(sDOLA_price_df)

In [ ]:
px.line(
    10_000
    * (sDOLA_price_df["sDOLA_spot_price"] - sDOLA_price_df["sDOLA_exchange_rate"])
    / sDOLA_price_df["sDOLA_exchange_rate"],
    title="sDOLA exchange rate vs spot price bps difference",
)

In [ ]:
sDOLA_price_df[sDOLA_price_df["sDOLA_spot_price"] > 0]

In [ ]:
auto_usd_root_price_oracle = "0xdB8747a396D75D576Dc7a10bb6c8F02F4a3C20f1"
dola = "0x865377367054516e17014CcdED1e7d814EDC9ce4"
sdola = "0xb45ad160634c528Cc3D2926d9807104FA3157305"

sdola_exchange_rate = Call(
    sdola,
    ["convertToAssets(uint256)(uint256)", int(1e18)],
    [("sdola_exchange_rate", safe_normalize_with_bool_success)],
)

sdola_spot_price_call = Call(
    auto_usd_root_price_oracle,
    ["getSpotPriceInQuote(address,address,address)(uint256)", sdola, sdola, dola],
    [("sdola_spot_price", safe_normalize_with_bool_success)],
)

autoDOLA_solver = "0x2795B6a09df08891EF21f7153f4799cFBe3Ae6b6"

dola_autopool_bal_call = Call(
    dola,
    ["balanceOf(address)(uint256)", AUTO_DOLA.autopool_eth_addr],
    [("dola_autopool_bal", safe_normalize_with_bool_success)],
)

dola_solver_bal_call = Call(
    dola,
    ["balanceOf(address)(uint256)", autoDOLA_solver],
    [("dola_solver_bal", safe_normalize_with_bool_success)],
)

sdola_autopool_bal_call = Call(
    sdola,
    ["balanceOf(address)(uint256)", AUTO_DOLA.autopool_eth_addr],
    [("sdola_autopool_bal", safe_normalize_with_bool_success)],
)

sdola_solver_bal_call = Call(
    sdola,
    ["balanceOf(address)(uint256)", autoDOLA_solver],
    [("sdola_solver_bal", safe_normalize_with_bool_success)],
)

block_number = 22621780

before = get_state_by_one_block(
    [
        sdola_exchange_rate,
        sdola_spot_price_call,
        dola_autopool_bal_call,
        dola_solver_bal_call,
        sdola_autopool_bal_call,
        sdola_solver_bal_call,
    ],
    block_number - 1,
    ETH_CHAIN,
)

after = get_state_by_one_block(
    [
        sdola_exchange_rate,
        sdola_spot_price_call,
        dola_autopool_bal_call,
        dola_solver_bal_call,
        sdola_autopool_bal_call,
        sdola_solver_bal_call,
    ],
    block_number,
    ETH_CHAIN,
)

{"before": before, "after": after}

In [ ]:

        for (pool_address, token_address) in zip(pool_token_addresses["pool"], pool_token_addresses["token_address"])

In [ ]:
df[["spot_value_in_solver_change", "spot_swap_cost"]].describe()

In [ ]:
df["spot_swap_cost_less_solver"] = df["spot_swap_cost"] - df["spot_value_in_solver_change"]

px.scatter(df, x="spot_swap_cost", y="spot_swap_cost_less_solver")

In [ ]:
px.histogram(df["spot_swap_cost_less_solver"])